In [1]:
# !pip install transformers[torch]
# !pip install sentence_transformers

In [2]:
from sentence_transformers import SentenceTransformer
import json
import sys
import torch
import requests

In [3]:
device = 'cpu'
distilroberta = SentenceTransformer('stsb-distilroberta-base-v2', device=device)
distilroberta.max_seq_length = 512
usenc = SentenceTransformer('distiluse-base-multilingual-cased-v1', device=device)
usenc.max_seq_length = 512

In [4]:
seq = "13 cases of measles, 2 new cases of measles were contracted from abroad Published Sunday, February 24, 2019 12:57PM PST  Last Updated Sunday, February 24, 2019 8:06PM PST  There are now 13 people with the measles, according to health officials in Vancouver. Vancouver Coastal Health said the two new cases are not related to the outbreak involving two French language schools. The health authority said the two people were infected while they were travelling, but has only specified that one of them went to the Philippines. A spokesperson with the Ministry of Health confirmed to CTV News the second person had also travelled to the island nation.  VCH said the first measles patient arrived at the YVR on Feb. 11 on Philippine Airlines flight PR 116 from Manila and had a one-night layover. The patient left Vancouver to Edmonton on Air Canada Flight AC236 on Feb. 12. \"The safety and security of our passengers, employees and community is our top priority. We recommend anyone who was on the flights or in the terminal at YVR on those days and may be experiencing symptoms to contact 811 to talk to a nurse,\" YVR spokesperson Chris Devauld said in an e-mail. The second measles patient is a resident of the Lower Mainland and was infectious when they went to a restaurant and mall in Richmond, according to the health authority. VCH identified several places where people may have been exposed to the highly contagious virus, including: YVR, international arrivals and customs area between 4 and 7 p.m. on Feb. 11 Shuttle bus from YVR to La Quinta Inn in Richmond between 5 to 7 p.m. on Feb. 11 Shuttle bus from YVR to La Quinta Inn in Richmond between 8 and 10:30 a.m. on Feb. 12 YVR, Air Canada domestic departures area between 9 a.m. and noon on Feb. 12 Little Onpin Restaurant in Richmond between 6 and 10 p.m. on Feb. 16 Toys R Us at Lansdowne Centre between 11 a.m. and 9 p.m. on Feb. 17 Toys R Us at Lansdowne Centre between 11 a.m. and 9 p.m. on Feb. 18 'We can do better'  Health officials said those who have never had the measles or did not have the two doses of the vaccine are at the highest risk of contracting the virus. They encourage people to get fully immunized, adding the shot is free for everyone. Health Minister Adrian Dix assures all British Columbians that there is enough vaccine to deal with the demand. \"Frankly, people shouldn't be getting measles in the 21st century in British Columbia,\" he said. \"While our healthcare system is doing very well, I think can we do better and we will do better.\" Health Minister Dix says BC is in communications with other provinces and Washington State about the measles outbreak. Says 12 cases is too many and again implores people to get vaccinated @CTVVancouverpic.twitter.com/c1NjsbpeNi Other cases linked to French schools  On Friday, Vancouver Coastal Health announced there was a total of 10 cases that were \"very clearly associated\" with the schools, Ecole Secondaire Jules-Verne and Ecole Rose-Des-Vents. Officials said 33 students at the two schools have been ordered to stay home because they haven't provided proof of measles vaccination.  The other three cases were contracted while travelling and one unrelated case was acquired locally, according to VCH. Health officials are reminding people they can get vaccinations by visiting their family doctor, a walk-in clinic or urgent primary care centre, or a pharmacy for adults or children over five years old. Calling ahead is also recommended."

In [5]:
distilroberta_embedding = distilroberta.encode(seq)
use_embedding = usenc.encode(seq)

In [6]:
query_distilroberta = {"size":50, 
         "_source": "content_t",
         "query": {
            "script_score": {
              "query" : {
                "match_all" : {}
                },
              "script": {
                "source": "(1.0+cosineSimilarity(params.queryVector, 'distilroberta_embedding'))",
                "params": {
                  "queryVector": distilroberta_embedding.tolist()
                    }
                }
            }
          }
        }

In [7]:
query_usenc = {"size":50, 
         "_source": "content_t",
         "query": {
            "script_score": {
              "query" : {
                "match_all" : {}
                },
              "script": {
                "source": "(1.0+cosineSimilarity(params.queryVector, 'use_embedding'))",
                "params": {
                  "queryVector": use_embedding.tolist()
                    }
                }
            }
          }
        }

In [10]:
response1 = requests.get('http://localhost:9200/posts/_search', json=query_distilroberta)
print(json.dumps(response1.json(), indent=1))

{
 "took": 3850,
 "timed_out": false,
 "_shards": {
  "total": 1,
  "successful": 1,
  "skipped": 0,
  "failed": 0
 },
 "hits": {
  "total": {
   "value": 10000,
   "relation": "gte"
  },
  "max_score": 2.0,
  "hits": [
   {
    "_index": "posts",
    "_type": "_doc",
    "_id": "43516898814867403655109610208249355561",
    "_score": 2.0,
    "_source": {
     "content_t": "13 cases of measles, 2 new cases of measles were contracted from abroad Published Sunday, February 24, 2019 12:57PM PST  Last Updated Sunday, February 24, 2019 8:06PM PST  There are now 13 people with the measles, according to health officials in Vancouver. Vancouver Coastal Health said the two new cases are not related to the outbreak involving two French language schools. The health authority said the two people were infected while they were travelling, but has only specified that one of them went to the Philippines. A spokesperson with the Ministry of Health confirmed to CTV News the second person had also travel

In [11]:
response2 = requests.get('http://localhost:9200/posts/_search', json=query_usenc)
print(json.dumps(response2.json(), indent=1))

{
 "took": 2608,
 "timed_out": false,
 "_shards": {
  "total": 1,
  "successful": 1,
  "skipped": 0,
  "failed": 0
 },
 "hits": {
  "total": {
   "value": 10000,
   "relation": "gte"
  },
  "max_score": 2.0,
  "hits": [
   {
    "_index": "posts",
    "_type": "_doc",
    "_id": "43516898814867403655109610208249355561",
    "_score": 2.0,
    "_source": {
     "content_t": "13 cases of measles, 2 new cases of measles were contracted from abroad Published Sunday, February 24, 2019 12:57PM PST  Last Updated Sunday, February 24, 2019 8:06PM PST  There are now 13 people with the measles, according to health officials in Vancouver. Vancouver Coastal Health said the two new cases are not related to the outbreak involving two French language schools. The health authority said the two people were infected while they were travelling, but has only specified that one of them went to the Philippines. A spokesperson with the Ministry of Health confirmed to CTV News the second person had also travel

In [12]:
from sentence_transformers import CrossEncoder
model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2', max_length=512)

In [51]:
import numpy as np
search = seq

In [30]:
scores1 = model.predict([(search, document["_source"]["content_t"]) for document in response1.json()["hits"]["hits"]])

In [46]:
print(json.dumps(np.flip(np.array(response1.json()["hits"]["hits"])[scores1.argsort()]).tolist(), indent=1))

[
 {
  "_index": "posts",
  "_type": "_doc",
  "_id": "43516898814867403655109610208249355561",
  "_score": 2.0,
  "_source": {
   "content_t": "13 cases of measles, 2 new cases of measles were contracted from abroad Published Sunday, February 24, 2019 12:57PM PST  Last Updated Sunday, February 24, 2019 8:06PM PST  There are now 13 people with the measles, according to health officials in Vancouver. Vancouver Coastal Health said the two new cases are not related to the outbreak involving two French language schools. The health authority said the two people were infected while they were travelling, but has only specified that one of them went to the Philippines. A spokesperson with the Ministry of Health confirmed to CTV News the second person had also travelled to the island nation.  VCH said the first measles patient arrived at the YVR on Feb. 11 on Philippine Airlines flight PR 116 from Manila and had a one-night layover. The patient left Vancouver to Edmonton on Air Canada Flight AC

In [47]:
scores2 = model.predict([(search, document["_source"]["content_t"]) for document in response2.json()["hits"]["hits"]])

In [48]:
print(json.dumps(np.flip(np.array(response2.json()["hits"]["hits"])[scores2.argsort()]).tolist(), indent=1))

[
 {
  "_index": "posts",
  "_type": "_doc",
  "_id": "43516898814867403655109610208249355561",
  "_score": 2.0,
  "_source": {
   "content_t": "13 cases of measles, 2 new cases of measles were contracted from abroad Published Sunday, February 24, 2019 12:57PM PST  Last Updated Sunday, February 24, 2019 8:06PM PST  There are now 13 people with the measles, according to health officials in Vancouver. Vancouver Coastal Health said the two new cases are not related to the outbreak involving two French language schools. The health authority said the two people were infected while they were travelling, but has only specified that one of them went to the Philippines. A spokesperson with the Ministry of Health confirmed to CTV News the second person had also travelled to the island nation.  VCH said the first measles patient arrived at the YVR on Feb. 11 on Philippine Airlines flight PR 116 from Manila and had a one-night layover. The patient left Vancouver to Edmonton on Air Canada Flight AC